In [1]:
import numpy as np
import pandas as pd

In [2]:
with open('./eval.txt', 'r') as f:
    results = f.readlines()

# Remove any non-result lines from the eval file, and split the lines on the tab character
# (results have format: model_name\tdataset_name\tmetric_name\tmetric_value)
results = [r.replace('\n','').split('\t') for r in results if '\t' in r]

In [3]:
def remove_underscore_after(val): # Remove underscore, and keep the part after the underscore
    return val.split('_')[-1]
def remove_underscore_before(val): # Remove underscore, and keep the part before the underscore
    return val.split('_')[0]
def map2d(func, grid): # Mapping for 2d arrays, from: https://stackoverflow.com/questions/70742445/elegant-map-over-2d-list
    return [[func(value) for value in row] for row in grid]

df = pd.DataFrame(results, columns = ['model', 'dataset', 'metric', 'value'])
df['value'] = pd.to_numeric(df['value'])
df['model'] = df['model'].map(lambda x: '-'.join(x.split('-')[2:]))
models = df['model'].str.split('-').tolist()

# Remove all underscores from our 2d list, keep one list of the param names and one with param vals
model_names_list = map2d(remove_underscore_before, models)
model_names = np.array(model_names_list)
model_val_list = map2d(remove_underscore_after, models)
model_vals = np.array(model_val_list)

In [4]:
model_vals.shape, model_names.shape

((1989, 12), (1989, 12))

In [5]:
# Splitting model name into more columns
for i in range(model_names.shape[-1]):
    name = model_names[0][i]
    val = model_vals[:,i]
    df[name] = val
    try:
       df[name] = pd.to_numeric(df[name]) 
    except:
        pass

cols = sorted(df.columns.tolist())
# The ratio have the format: 'ratio_xxx' where xxx is some float.
# We only want that float value, and need to convert it from string to float.
# df['ratio'] = pd.to_numeric(df['ratio'])
display(df)

# Group by all columns except the performance values, then compute mean, std and number of model runs for the performance
df_grouped = df.groupby(list(set(cols)-set(['value']))).agg({'value':['mean', 'std', 'count']})

,model,dataset,metric,value,data,ALL,ratio,method,kw,AL,PL,vit,epochs,lr,bs
0,RN50,RSICD-CLS,zeroshot-val-top1,3.93,RS,ALL,0.1,base,none,False,None,False,5,0.0005,64
1,RN50,UCM-CLS,zeroshot-val-top1,5.71,RS,ALL,0.1,base,none,False,None,False,5,0.0005,64
2,RN50,RSICD,image_to_text_R@1,0.09,RS,ALL,0.1,base,none,False,None,False,5,0.0005,64
3,RN50,RSICD,image_to_text_R@5,0.55,RS,ALL,0.1,base,none,False,None,False,5,0.0005,64
4,RN50,RSICD,image_to_text_R@10,1.01,RS,ALL,0.1,base,none,False,None,False,5,0.0005,64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1984,RN50,RSICD,image_to_text_R@5,0.64,RS,ALL,0.1,base,none,False,None,False,35,0.0005,128
1985,RN50,RSICD,image_to_text_R@10,1.74,RS,ALL,0.1,base,none,False,None,False,35,0.0005,128
1986,RN50,RSICD,text_to_image_R@1,0.18,RS,ALL,0.1,base,none,False,None,False,35,0.0005,128
1987,RN50,RSICD,text_to_image_R@5,1.01,RS,ALL,0.1,base,none,False,None,False,35,0.0005,128


In [6]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df_grouped)

value  \
                                                                                                         mean   
data kw   PL   ratio lr       model AL    metric             dataset   bs  method vit   ALL epochs              
RS   none None 0.1   0.000005 RN50  False image_to_text_R@1  RSICD     64  base   False ALL 5        1.884000   
                                                                                            15       2.540000   
                                                                                            25       2.580000   
                                                                       128 base   False ALL 5        1.664000   
                                                                                            15       2.888000   
                                                                                            25       2.650000   
                                                             Sydney    64  base   False ALL 5        2.496000   
                                                                                            15       2.898000   
                                                                                            25       3.380000   
                                                                       128 base   False ALL 5        1.730000   
                                                                                            15       2.978000   
                                                                                            25       3.462000   
                                                             UCM       64  base   False ALL 5        1.154000   
                                                                                            15       1.562000   
                                                                                            25       2.002000   
                                                                       128 base   False ALL 5        0.772000   
                                                                                            15       1.418000   
                                                                                            25       2.068000   
                                          image_to_text_R@10 RSICD     64  base   False ALL 5       12.522000   
                                                                                            15      22.304000   
                                                                                            25      22.854000   
                                                                       128 base   False ALL 5        9.832000   
                                                                                            15      19.652000   
                                                                                            25      21.260000   
                                                             Sydney    64  base   False ALL 5       13.160000   
                                                                                            15      19.314000   
                                                                                            25      19.196000   
                                                                       128 base   False ALL 5       12.516000   
                                                                                            15      16.982000   
                                                                                            25      18.794000   
                                                             UCM       64  base   False ALL 5        9.014000   
                                                                                            15      14.416000   
                                                                                            25      15.346000   
                                    

In [7]:
df_grouped.to_csv('eval.csv')

In [9]:
df_2 = df_grouped.reset_index()
# df_2 = df_2[(df_2['method'] == 'ours') & (df_2['ratio'] == 0.2) & (df_2['PL'] == 'ot.image')] # (df_2['metric'] == 'image_to_text_R@5') &
df_2 = df_2[(df_2['epochs'] > 10)] # &(df_2['lr']==5e-5)
df_2 = df_2.sort_values(['metric', 'dataset', ('value', 'mean')])
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_2)

data    kw    PL ratio        lr model     AL              metric  \
                                                                        
245   RS  none  None   0.1  0.000500  RN50  False   image_to_text_R@1   
246   RS  none  None   0.1  0.000500  RN50  False   image_to_text_R@1   
247   RS  none  None   0.1  0.000500  RN50  False   image_to_text_R@1   
241   RS  none  None   0.1  0.000500  RN50  False   image_to_text_R@1   
242   RS  none  None   0.1  0.000500  RN50  False   image_to_text_R@1   
243   RS  none  None   0.1  0.000500  RN50  False   image_to_text_R@1   
1     RS  none  None   0.1  0.000005  RN50  False   image_to_text_R@1   
2     RS  none  None   0.1  0.000005  RN50  False   image_to_text_R@1   
5     RS  none  None   0.1  0.000005  RN50  False   image_to_text_R@1   
4     RS  none  None   0.1  0.000005  RN50  False   image_to_text_R@1   
125   RS  none  None   0.1  0.000050  RN50  False   image_to_text_R@1   
122   RS  none  None   0.1  0.000050  RN50  False   image_to_text_R@1   
124   RS  none  None   0.1  0.000050  RN50  False   image_to_text_R@1   
121   RS  none  None   0.1  0.000050  RN50  False   image_to_text_R@1   
254   RS  none  None   0.1  0.000500  RN50  False   image_to_text_R@1   
253   RS  none  None   0.1  0.000500  RN50  False   image_to_text_R@1   
255   RS  none  None   0.1  0.000500  RN50  False   image_to_text_R@1   
250   RS  none  None   0.1  0.000500  RN50  False   image_to_text_R@1   
249   RS  none  None   0.1  0.000500  RN50  False   image_to_text_R@1   
251   RS  none  None   0.1  0.000500  RN50  False   image_to_text_R@1   
7     RS  none  None   0.1  0.000005  RN50  False   image_to_text_R@1   
10    RS  none  None   0.1  0.000005  RN50  False   image_to_text_R@1   
127   RS  none  None   0.1  0.000050  RN50  False   image_to_text_R@1   
128   RS  none  None   0.1  0.000050  RN50  False   image_to_text_R@1   
8     RS  none  None   0.1  0.000005  RN50  False   image_to_text_R@1   
11    RS  none  None   0.1  0.000005  RN50  False   image_to_text_R@1   
130   RS  none  None   0.1  0.000050  RN50  False   image_to_text_R@1   
131   RS  none  None   0.1  0.000050  RN50  False   image_to_text_R@1   
262   RS  none  None   0.1  0.000500  RN50  False   image_to_text_R@1   
261   RS  none  None   0.1  0.000500  RN50  False   image_to_text_R@1   
257   RS  none  None   0.1  0.000500  RN50  False   image_to_text_R@1   
258   RS  none  None   0.1  0.000500  RN50  False   image_to_text_R@1   
263   RS  none  None   0.1  0.000500  RN50  False   image_to_text_R@1   
259   RS  none  None   0.1  0.000500  RN50  False   image_to_text_R@1   
16    RS  none  None   0.1  0.000005  RN50  False   image_to_text_R@1   
13    RS  none  None   0.1  0.000005  RN50  False   image_to_text_R@1   
14    RS  none  None   0.1  0.000005  RN50  False   image_to_text_R@1   
17    RS  none  None   0.1  0.000005  RN50  False   image_to_text_R@1   
133   RS  none  None   0.1  0.000050  RN50  False   image_to_text_R@1   
136   RS  none  None   0.1  0.000050  RN50  False   image_to_text_R@1   
134   RS  none  None   0.1  0.000050  RN50  False   image_to_text_R@1   
137   RS  none  None   0.1  0.000050  RN50  False   image_to_text_R@1   
269   RS  none  None   0.1  0.000500  RN50  False  image_to_text_R@10   
270   RS  none  None   0.1  0.000500  RN50  False  image_to_text_R@10   
265   RS  none  None   0.1  0.000500  RN50  False  image_to_text_R@10   
271   RS  none  None   0.1  0.000500  RN50  False  image_to_text_R@10   
266   RS  none  None   0.1  0.000500  RN50  False  image_to_text_R@10   
267   RS  none  None   0.1  0.000500  RN50  False  image_to_text_R@10   
22    RS  none  None   0.1  0.000005  RN50  False  image_to_text_R@10   
23    RS  none  None   0.1  0.000005  RN50  False  image_to_text_R@10   
19    RS  none  None   0.1  0.000005  RN50  False  image_to_text_R@10   
20    RS  none  None   0.1  0.000005  RN50  False  image_to_text_R@10   
142   RS  none  None   0.1  0.000050  RN50  False  image_to_te